# Baseline FLAIR Testing & Evaluation


## 1. Setup & Installation


In [ ]:
!pip install -q nnunetv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66

## 2. Environment Configuration


In [ ]:
import os
from pathlib import Path
import shutil, nibabel as nib, json, pandas as pd, numpy as np
from scipy.spatial.distance import directed_hausdorff

# USER CONFIG
DATASET_ID = 2  # <--- ID of the dataset to use
DATASET_NAME = "BonnFCD_FLAIR"  # <--- Name of the dataset
CFG = "3d_fullres"  # <--- nnU-Net configuration (e.g., 2d, 3d_fullres)
FOLD = "0"   # <--- Fold number to use (0, 1, 2, 3, 4)
CKPT = "checkpoint_best.pth"  # <--- Checkpoint file name (e.g., checkpoint_best.pth)
TRAINER = "nnUNetTrainer"  # Your custom trainer name

DATASET_DIR = f"Dataset{DATASET_ID:03d}_{DATASET_NAME}"
INPUT_ROOT = Path("/kaggle/input/flair-training-baseline-dataset-1")  # <--- Root directory for input data
RAW_SRC = INPUT_ROOT / "nnUNet_raw_data" / DATASET_DIR

# nnU-Net environment variables
os.environ["nnUNet_raw"] = str(INPUT_ROOT / "nnUNet_raw_data")
os.environ["nnUNet_preprocessed"] = str(INPUT_ROOT / "nnUNet_preprocessed")
os.environ["nnUNet_results"] = str(INPUT_ROOT / "nnUNet_results")

print(f"✅ Environment configured for dataset: {DATASET_DIR}")
print(f"📁 Input root: {INPUT_ROOT}")
print(f"📁 Raw data: {RAW_SRC}")

# Use imagesTs and labelsTs directly from INPUT_ROOT
IMAGES_TS = RAW_SRC / "imagesTs"
LABELS_TS = RAW_SRC / "labelsTs"

print(f"📁 Test images: {IMAGES_TS}")
print(f"📁 Test labels: {LABELS_TS}")

# Verify folders exist
if not IMAGES_TS.exists():
    raise FileNotFoundError(f"imagesTs folder not found at {IMAGES_TS}")
if not LABELS_TS.exists():
    raise FileNotFoundError(f"labelsTs folder not found at {LABELS_TS}")

print(f"✅ Found {len(list(IMAGES_TS.glob('*.nii*')))} images")
print(f"✅ Found {len(list(LABELS_TS.glob('*.nii*')))} labels")

✅ Environment configured for dataset: Dataset002_BonnFCD_FLAIR
📁 Input root: /kaggle/input/flair-training-baseline-dataset-1
📁 Raw data: /kaggle/input/flair-training-baseline-dataset-1/nnUNet_raw_data/Dataset002_BonnFCD_FLAIR
📁 Test images: /kaggle/input/flair-training-baseline-dataset-1/nnUNet_raw_data/Dataset002_BonnFCD_FLAIR/imagesTs
📁 Test labels: /kaggle/input/flair-training-baseline-dataset-1/nnUNet_raw_data/Dataset002_BonnFCD_FLAIR/labelsTs
✅ Found 28 images
✅ Found 28 labels


## 3. Inference


In [ ]:
INP = IMAGES_TS
OUT = Path(f"/kaggle/working/preds_D{DATASET_ID:03d}_{CFG}_fold{FOLD}_best")
OUT.mkdir(parents=True, exist_ok=True)

print("🔍 Running inference ...")
print(f"   Input: {INP}")
print(f"   Output: {OUT}")
print(f"   Trainer: {TRAINER}")
print(f"   Fold: {FOLD}")

!nnUNetv2_predict -i "$INP" -o "$OUT" -d "$DATASET_ID" -c "$CFG" -f "$FOLD" -tr "$TRAINER" -chk "$CKPT"
print(f"✅ Predictions saved to: {OUT}")

🔍 Running inference ...
   Input: /kaggle/input/flair-training-baseline-dataset-1/nnUNet_raw_data/Dataset002_BonnFCD_FLAIR/imagesTs
   Output: /kaggle/working/preds_D002_3d_fullres_fold0_best
   Trainer: nnUNetTrainer
   Fold: 0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 28 cases in the source folder
I am processing 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 28 cases that I would like to predict

Predicting sub-00004:
perform_everything_on_device: True
100%|███████████████████████████████████████████| 27/27 [00:55<00:00,  2.07s/it]
sending off prediction to background worker for resampling and ex

## 4. Evaluation Metrics


In [ ]:
def dice_score(gt, pred):
    intersection = np.sum((gt > 0) & (pred > 0))
    total = np.sum(gt > 0) + np.sum(pred > 0)
    return 2 * intersection / total if total > 0 else 1.0

def iou_score(gt, pred):
    intersection = np.sum((gt > 0) & (pred > 0))
    union = np.sum((gt > 0) | (pred > 0))
    return intersection / union if union > 0 else 1.0

def precision_recall_f1(gt, pred):
    tp = np.sum((gt > 0) & (pred > 0))
    fp = np.sum((gt == 0) & (pred > 0))
    fn = np.sum((gt > 0) & (pred == 0))
    precision = tp / (tp + fp) if (tp + fp) > 0 else 1.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 1.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1

results = []
print(f"\n🔎 Looking for labels in: {LABELS_TS}")
print(f"🔎 Looking for predictions in: {OUT}")

label_files = list(sorted(LABELS_TS.glob("*.nii*")))
pred_files = list(sorted(OUT.glob("*.nii*")))

print(f"\n📋 Found {len(label_files)} label files")
print(f"📋 Found {len(pred_files)} prediction files")

if len(label_files) == 0:
    print("❌ ERROR: No label files found!")
if len(pred_files) == 0:
    print("❌ ERROR: No prediction files found!")

for lbl_file in label_files:
    stem = lbl_file.stem.replace('.nii', '')  # Handle both .nii and .nii.gz

    # Try to find matching prediction with different extensions
    pred_file = None
    for ext in ['.nii', '.nii.gz']:
        potential_pred = OUT / f"{stem}{ext}"
        if potential_pred.exists():
            pred_file = potential_pred
            break

    if pred_file is None:
        print(f"⚠️  No prediction found for {stem}")
        continue

    print(f"✓ Processing {stem}")
    gt = nib.load(str(lbl_file)).get_fdata()
    pred = nib.load(str(pred_file)).get_fdata()

    dice = dice_score(gt, pred)
    iou = iou_score(gt, pred)
    prec, rec, f1 = precision_recall_f1(gt, pred)

    results.append({
        "Case": stem,
        "Dice": dice,
        "IoU": iou,
    })


🔎 Looking for labels in: /kaggle/input/flair-training-baseline-dataset-1/nnUNet_raw_data/Dataset002_BonnFCD_FLAIR/labelsTs
🔎 Looking for predictions in: /kaggle/working/preds_D002_3d_fullres_fold0_best

📋 Found 28 label files
📋 Found 28 prediction files
✓ Processing sub-00004
✓ Processing sub-00006
✓ Processing sub-00009
✓ Processing sub-00020
✓ Processing sub-00032
✓ Processing sub-00034
✓ Processing sub-00043
✓ Processing sub-00048
✓ Processing sub-00061
✓ Processing sub-00062
✓ Processing sub-00064
✓ Processing sub-00066
✓ Processing sub-00074
✓ Processing sub-00092
✓ Processing sub-00095
✓ Processing sub-00099
✓ Processing sub-00100
✓ Processing sub-00103
✓ Processing sub-00107
✓ Processing sub-00108
✓ Processing sub-00121
✓ Processing sub-00125
✓ Processing sub-00128
✓ Processing sub-00134
✓ Processing sub-00136
✓ Processing sub-00142
✓ Processing sub-00144
✓ Processing sub-00145


## 5. Results Analysis


In [ ]:
if len(results) == 0:
    print("\n❌ ERROR: No results computed. Check that predictions match labels.")
else:
    df = pd.DataFrame(results).round(4)
    df.to_csv("/kaggle/working/full_metrics_per_case.csv", index=False)

    print(f"\n{df.to_string()}")
    print("\n" + "="*60)
    print("📊 SUMMARY STATISTICS")
    print("="*60)
    print(f"📊 Mean Dice:       {df['Dice'].mean():.4f}")
    print(f"📊 Mean IoU:        {df['IoU'].mean():.4f}")
    # print(f"🎯 Mean Precision:  {df['Precision'].mean():.4f}")
    # print(f"🎯 Mean Recall:     {df['Recall'].mean():.4f}")
    # print(f"🏅 Mean F1-score:   {df['F1'].mean():.4f}")
    print("="*60)
    print("💾 Saved → /kaggle/working/full_metrics_per_case.csv")


         Case    Dice     IoU
0   sub-00004  0.0000  0.0000
1   sub-00006  0.0000  0.0000
2   sub-00009  0.0000  0.0000
3   sub-00020  0.5358  0.3659
4   sub-00032  0.0000  0.0000
5   sub-00034  0.0000  0.0000
6   sub-00043  0.0000  0.0000
7   sub-00048  0.0000  0.0000
8   sub-00061  0.7637  0.6178
9   sub-00062  0.0000  0.0000
10  sub-00064  0.0750  0.0390
11  sub-00066  0.0904  0.0473
12  sub-00074  0.0000  0.0000
13  sub-00092  0.5882  0.4166
14  sub-00095  0.0213  0.0108
15  sub-00099  0.1999  0.1110
16  sub-00100  0.0000  0.0000
17  sub-00103  0.0000  0.0000
18  sub-00107  0.0000  0.0000
19  sub-00108  0.0000  0.0000
20  sub-00121  0.0000  0.0000
21  sub-00125  0.4129  0.2601
22  sub-00128  0.5382  0.3681
23  sub-00134  0.1966  0.1090
24  sub-00136  0.0000  0.0000
25  sub-00142  0.4927  0.3268
26  sub-00144  0.3749  0.2307
27  sub-00145  0.6552  0.4872

📊 SUMMARY STATISTICS
📊 Mean Dice:       0.1766
📊 Mean IoU:        0.1211
💾 Saved → /kaggle/working/full_metrics_per_case.csv
